In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [7]:
# Each image FashionMIST is a grayscale 28x28 (total 784 pixels) image, each pixel has a single value between  
# 0 and 255, the higher the darker the pixel. The training and test datasets have 785 columns, the first is 
# the class label (what kind of cloth it is).
training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

In [12]:
# Each item in the dataloader iterable will return a batch of 64 features and labels (images).
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# X: features, y: label
for X, y in test_dataloader:
    print(f"Shape of X [N(#image), C(color), H(height), W(width)]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N(#image), C(color), H(height), W(width)]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [23]:
# our neural net architecture is fully connected, the input layer has 28*28 nodes, with one 512 nodes hidden layer 
# with ReLU activation function, the output layer has 10 nodes, each representing the probability for each label.
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        # make tensors 1d
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNet().to(device)
print(model)

NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
# the loss function and the optimzer
lossfn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=1e-3)

In [19]:
def train(dataloader, model, lossfn, optim):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = lossfn(pred, y)
        
        # backpropagation
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [25]:
def test(dataloader, model, lossfn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += lossfn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg Loss: {test_loss:>8f} \n")

In [26]:
epochs = 5
for e in range(epochs):
    print(f"Epoch: {e+1}")
    print('-'*30)
    train(train_dataloader, model, lossfn, optim)
    test(test_dataloader, model, lossfn)
print("Finished!")

Epoch: 1
------------------------------
loss: 2.314366 [    0/60000]
loss: 2.309913 [ 6400/60000]
loss: 2.316415 [12800/60000]
loss: 2.306095 [19200/60000]
loss: 2.300427 [25600/60000]
loss: 2.295782 [32000/60000]
loss: 2.298127 [38400/60000]
loss: 2.302067 [44800/60000]
loss: 2.312053 [51200/60000]
loss: 2.288653 [57600/60000]
Test Error: 
 Accuracy: 13.6%, Avg Loss: 2.303258 

Epoch: 2
------------------------------
loss: 2.314366 [    0/60000]
loss: 2.309913 [ 6400/60000]
loss: 2.316415 [12800/60000]
loss: 2.306095 [19200/60000]
loss: 2.300427 [25600/60000]
loss: 2.295782 [32000/60000]
loss: 2.298127 [38400/60000]
loss: 2.302067 [44800/60000]
loss: 2.312053 [51200/60000]
loss: 2.288653 [57600/60000]
Test Error: 
 Accuracy: 13.6%, Avg Loss: 2.303258 

Epoch: 3
------------------------------
loss: 2.314366 [    0/60000]
loss: 2.309913 [ 6400/60000]
loss: 2.316415 [12800/60000]
loss: 2.306095 [19200/60000]
loss: 2.300427 [25600/60000]
loss: 2.295782 [32000/60000]
loss: 2.298127 [38400/

In [28]:
# serialize the internal state dictionary containing the model parameters
torch.save(model.state_dict(), "model_FashionMNIST.pth")